## This is the first in a series of examples provided to demonstrate the use of SAS Visual Forecasting actions to compose a program that             
- loads transactional data,                                                          
- accumulates the transactional data to time series,                                                    
- optionally promotes the time series for pick up in other applications running on CAS                           
                                  
                                                                          
The programs are written to execute in the CAS in-memory distributed computing engine in the SAS Viya environment.                            
                                                                          
This first example shows how to load local data into CAS             

## load packages

In [ ]:
from swat import *

# CAS server connection details

In [ ]:
cashost='localhost'
casport=5570
casauth='~/.authinfo'

# Start CAS session

In [ ]:
sess = CAS(cashost, casport, authinfo=casauth, caslib="casuser")

# Details for local data to be loaded into CAS

In [ ]:
indata_dir="/opt/sasinside/DemoData"
indata="pricedata"

# load the table action set

In [ ]:
sess.loadactionset(actionset="table")
#sess.actionsetinfo(all='true')

# load the timedata action set

In [ ]:
sess.loadactionset(actionset="timedata")

# load data into CAS

## For execution in the CAS engine, data must be loaded from the local data set to a CAS table.   
## This code first checks to see if the specified CAS table exists and then loads data from local ## data sets.      

In [ ]:
if not sess.table.tableExists(table=indata).exists:
    tbl = sess.upload_file(indata_dir+"/"+indata+".sas7bdat", casout={"name":indata})

In [ ]:
tbl.head(10)

# accumulate transactonal sale, price and discount series corresponding to productlines in pricedata to a monthly interval

In [ ]:
sess.timedata.timeSeries(
   table={"name":indata, "groupBy":{"regionName", "productLine"}},
   series= [{"name":"sale", "acc":"sum"}, {"name":"price", "acc":"avg"}, {"name":"discount", "acc":"avg"}],
   timeid='date',
   interval='month',
   casOut={"name":"pricemidpy", "replace":True}
   )

In [ ]:
sess.table.tableinfo(caslib='casuser')

# optional, promote the priceline timeseries for pickup in other CAS applications

In [ ]:
if not sess.table.tableExists(table='PRICEMIDPY').exists:
    sess.table.promote('PRICEMIDPY')

# end CAS session

In [ ]:
sess.close()